In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [4]:
from m5.functions import *
%aimport m5.functions

In [5]:
cal = pd.read_csv('../data/calendar.csv', parse_dates=['date'],                   
                  dtype={'event_name_1':'string', 'event_name_2':'string'})
sales = pd.read_csv('../data/sales_train_validation.csv', dtype = {'id':'string'})

In [6]:
sales

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,2,0,0,0,0,0,1,0,0,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,2,1,0,2,0,1,0,0,1,0
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [21]:
for i in ['event_name_1', 'event_name_2']:
    le = LabelEncoder()
    cal[i] = cal[i].apply(lambda x: '' if pd.isnull(x) else x)
    cal[i + '_ord'] = le.fit_transform(cal[i])
last_day = 1914 + 2*28
for day in range(1914, last_day):
    sales['d_' + str(day)] = 0 
sales['item_store_id'] = sales['item_id'] + '_' + sales['store_id']
ts_sales = sales.loc[:,'d_1':'d_' + str(last_day - 1)].T
ts_sales.columns = sales['item_store_id'].values
date_index = cal['date']
dates_list = date_index[0:last_day]
ts_sales = ts_sales.set_index([dates_list])
cal = cal.set_index([dates_list])
ts_sales = ts_sales.join(cal)
ts_sales.to_pickle('../ts_sales.pk')

In [ ]:
prices = pd.read_csv('../data/sell_prices.csv')
prices['item_store_id'] = prices['item_id'] + '_' + prices['store_id']
index_columns = dict()
for i in range(sales['item_store_id'].shape[0]):
    index_columns[sales['item_store_id'][i]] = i   
prices['item_store_id_ix'] = prices['item_store_id'].apply(lambda x: index_columns[x])

ts_prices_np = np.array([[0.0]*sales['item_store_id'].shape[0]]*date_index.shape[0])
for r in prices.itertuples():
    ts_prices_np[7*(r[3] - 11101):7*(r[3]+1 - 11101),r[6]] = r[4]
    
ts_prices = pd.DataFrame(ts_prices_np, columns=sales['item_store_id'], index=ts_sales_cal.index)
ts_prices.to_pickle('../ts_prices.pk')

In [23]:
print(sales.state_id.unique())
print(sales.item_id.unique())
print(sales.cat_id.unique())
print(sales.dept_id.unique())
print(sales.store_id.unique())

['CA' 'TX' 'WI']
['HOBBIES_1_001' 'HOBBIES_1_002' 'HOBBIES_1_003' ... 'FOODS_3_825'
 'FOODS_3_826' 'FOODS_3_827']
['HOBBIES' 'HOUSEHOLD' 'FOODS']
['HOBBIES_1' 'HOBBIES_2' 'HOUSEHOLD_1' 'HOUSEHOLD_2' 'FOODS_1' 'FOODS_2'
 'FOODS_3']
['CA_1' 'CA_2' 'CA_3' 'CA_4' 'TX_1' 'TX_2' 'TX_3' 'WI_1' 'WI_2' 'WI_3']


In [ ]:
# sold item_id
# sold in dept_id 
# sold cat_id 
# sold store_id
# item_id, dept_id, cat_id, store_id, state_id
# HOBBIES_1_008_CA_1 = HOBBIES_1_001, HOBBIES_1, HOBBIES, CA_1, CA

item_id = 'HOBBIES_1_008'
state_id = 'CA'

item = aggregate([item_id], ts_sales)